In [ ]:
# default_exp datasets.mts

# MTS Dataset
> data from the MTS Kion application on user interactions with content for a period of 6 months.

The presented dataset contains data on users and objects (series / movies), as well as on their interactions (content viewing by the user) from the Kion online cinema. Content view data collected for ~6 months, from 2021-03-13 to 2021-08-22 inclusive, and diluted with random noise. User and content IDs are anonymised.

### users.csv
This file contains information about users:
- user_id - User ID
- age - user's age group, string like "M_N".
 - 18_24 - from 18 to 24 years old inclusive
 - 25_34 - from 25 to 34 years old inclusive
 - 35_44 - from 35 to 44 years old inclusive
 - 45_54 - from 45 to 54 years old inclusive
 - 55_64 - from 55 to 64 years old inclusive
 - 65_inf - from 65 and older
- sex - user gender
 - M - man
 - F - woman
- income - user's income, string like "M_N
 - income_0_20
 - income_20_40
 - income_40_60
 - income_60_90  
 - income_90_150
 - income_150_inf
- kids_flg - flag "presence of a child

### items.csv
This file contains information about objects (movies/series):
- item_id - Content ID
- content_type - Type of content (movie, series)
- title - Title in Russian
- title_orig - original name
- genres - Genres from source (online movie theaters)
- countries - country
- for_kids - flag "content for children"
- age_rating - age rating
- studios - studios
- directors - directors
- actors - actors
- keywords - keywords
- description - description

### interactions.csv
This file contains information about user interactions with content:
- user_id - User ID
- item_id - Content ID
- last_watch_dt - Date last viewed
- total_dur - The total duration of all views of this content in seconds
- content_type - Type of content (movie, series)

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.nb_imports import *
from fastcore.test import *

In [ ]:
#export
from typing import Any, Iterable, List, Optional, Tuple, Union, Callable
import os

import pandas as pd
import numpy as np

from recohut.utils.common_utils import *
from recohut.datasets.bases.interactions import InteractionsDataset

In [ ]:
#export
class MTSDataset(InteractionsDataset):
    url_users = "https://github.com/RecoHut-Datasets/mts_kion/raw/v1/users.parquet.snappy"
    url_items = "https://github.com/RecoHut-Datasets/mts_kion/raw/v1/items.parquet.snappy"
    url_inter = "https://github.com/RecoHut-Datasets/mts_kion/raw/v1/interactions.parquet.snappy"

    def __init__(self, sample_frac=1, **kwargs):
        super().__init__(**kwargs)
        self.sample_frac = sample_frac

    @property
    def raw_file_names(self):
        return ['users.parquet.snappy',
                'items.parquet.snappy',
                'interactions.parquet.snappy']

    @property
    def processed_file_names(self):
        return ['users_processed.csv',
                'items_processed.csv',
                'interactions_processed.csv',
                'item_stats.csv']

    def download(self):
        _ = download_url(self.url_users, self.raw_dir)
        _ = download_url(self.url_items, self.raw_dir)
        _ = download_url(self.url_inter, self.raw_dir)

    def load_users_df(self):
        df = pd.read_parquet(self.raw_paths[0])
        return df

    def load_items_df(self):
        df = pd.read_parquet(self.raw_paths[1])
        return df

    def load_ratings_df(self):
        df = pd.read_parquet(self.raw_paths[2])
        df = df.sample(frac=sample_frac)
        return df

    @staticmethod
    def add_user_stats(interactions_df, users_df, split_name=''):
        """
        Computes user watches stats for particular interactions date split
        and adds them to users dataframe with specific name
        """
        user_watch_count_all = interactions_df[
            interactions_df['total_dur'] > 300].groupby(by='user_id')['item_id'].count()
        max_date_df = interactions_df['last_watch_dt'].max()
        user_watch_count_last_14 = interactions_df[
            (interactions_df['total_dur'] > 300) &
            (interactions_df['last_watch_dt'] >= (max_date_df - pd.Timedelta(days=14)))
            ].groupby(by='user_id')['item_id'].count()
        user_watch_count_all.name = split_name + "user_watch_cnt_all"
        user_watch_count_last_14.name = split_name + "user_watch_cnt_last_14"
        user_watches = pd.DataFrame(user_watch_count_all).join(user_watch_count_last_14,
                                                            how='outer')
        user_watches.fillna(0, inplace=True)
        cols = user_watches.columns
        user_watches[cols] = user_watches[cols].astype('int64')
        users_df = users_df.join(user_watches, on='user_id', how='outer')
        users_df[cols] = users_df[cols].fillna(0)
        users_df['age'] = users_df['age'].fillna('age_unknown')
        users_df['income'] = users_df['income'].fillna('income_unknown')
        users_df['sex'] = users_df['sex'].fillna('sex_unknown')
        users_df['kids_flg'] = users_df['kids_flg'].fillna(False)
        return users_df

    @staticmethod
    def add_item_watches_stats(interactions_df, items_df, item_stats):
        """
        Computes item watches stats for particular interactions date split
        and adds them to item_stats dataframe
        """

        def smooth(series, window_size, smoothing_func):
            """Computes smoothed interactions statistics for item"""
            series = np.array(series)
            ext = np.r_[2 * series[0] - series[window_size - 1::-1],
                        series,
                        2 * series[-1] - series[-1:-window_size:-1]]
            weights = smoothing_func(window_size)
            smoothed = np.convolve(weights / weights.sum(), ext, mode='same')
            return smoothed[window_size:-window_size + 1]

        def trend_slope(series, window_size=7, smoothing_func=np.hamming):
            """Computes trend slope for item interactions"""
            smoothed = smooth(series, window_size, smoothing_func)
            return smoothed[-1] - smoothed[-2]

        keep = item_stats.columns
        max_date = interactions_df['last_watch_dt'].max()
        cols = list(range(7))
        for col in cols:
            watches = interactions_df[
                interactions_df['last_watch_dt'] ==
                max_date - pd.Timedelta(days=6 - col)]
            item_stats = item_stats.join(
                watches.groupby('item_id')['user_id'].count(), lsuffix=col)
        item_stats.fillna(0, inplace=True)
        new_colnames = ['user_id' + str(i) for i in range(1, 7)] + ['user_id']
        trend_slope_to_row = lambda row: trend_slope(row[new_colnames],
                                                    window_size=7)
        item_stats['trend_slope'] = item_stats.apply(trend_slope_to_row,
                                                    axis=1)
        item_stats['watched_in_7_days'] = item_stats[new_colnames].apply(
            sum, axis=1)
        item_stats['watch_ts_quantile_95'] = 0
        item_stats['watch_ts_median'] = 0
        item_stats['watch_ts_std'] = 0
        for item_id in item_stats.index:
            watches = interactions_df[interactions_df['item_id'] == item_id]
            day_of_year = watches['last_watch_dt'].apply(
                lambda x: x.dayofyear).astype(np.int64)
            item_stats.loc[item_id, 'watch_ts_quantile_95'] = \
                day_of_year.quantile(q=0.95, interpolation='nearest')
            item_stats.loc[item_id, 'watch_ts_median'] = \
                day_of_year.quantile(q=0.5, interpolation='nearest')
            item_stats.loc[item_id, 'watch_ts_std'] = day_of_year.std()
        item_stats['watch_ts_quantile_95_diff'] = \
            max_date.dayofyear - item_stats['watch_ts_quantile_95']
        item_stats['watch_ts_median_diff'] = max_date.dayofyear - \
                                            item_stats['watch_ts_median']
        watched_all_time = interactions_df.groupby('item_id')['user_id'].count()
        watched_all_time.name = 'watched_in_all_time'
        item_stats = item_stats.join(watched_all_time, on='item_id', how='left')
        item_stats.fillna(0, inplace=True)
        added_cols = ['trend_slope',
                    'watched_in_7_days',
                    'watch_ts_quantile_95_diff',
                    'watch_ts_median_diff',
                    'watch_ts_std',
                    'watched_in_all_time']
        return item_stats[list(keep) + added_cols]

    @staticmethod
    def add_age_stats(interactions, item_stats, users_df):
        """
        Computes watchers age stats for items with particular interactions 
        date split and adds them to item_stats dataframe
        """
        item_stats.reset_index(inplace=True)
        interactions = interactions.set_index('user_id').join(
            users_df[['user_id', 'sex', 'age', 'income']].set_index('user_id'))
        interactions.reset_index(inplace=True)
        interactions['age_overall'] = interactions['age'].replace(
            to_replace={'age_18_24': 'less_35',
                        'age_25_34': 'less_35',
                        'age_35_44': 'over_35',
                        'age_45_54': 'over_35',
                        'age_65_inf': 'over_35',
                        'age_55_64': 'over_35'})
        age_stats = interactions.groupby('item_id')['age_overall'] \
            .value_counts(normalize=True)
        age_stats = pd.DataFrame(age_stats)
        age_stats.columns = ['value']
        age_stats.reset_index(inplace=True)
        age_stats.columns = ['item_id', 'age_overall', 'value']
        age_stats = age_stats.pivot(
            index='item_id', columns='age_overall', values='value').drop(
            'age_unknown', axis=1)
        age_stats.fillna(0, inplace=True)
        item_stats = item_stats.set_index('item_id').join(age_stats)
        item_stats[['less_35', 'over_35']] = item_stats[['less_35', 'over_35']] \
            .fillna(0)
        item_stats.rename(columns={'less_35': 'younger_35_fraction',
                                'over_35': 'older_35_fraction'},
                        inplace=True)
        return item_stats

    @staticmethod
    def add_sex_stats(interactions, item_stats, users_df):
        """
        Computes watchers sex stats for items with particular interactions date split
        and adds them to item_stats dataframe
        """
        item_stats.reset_index(inplace=True)
        interactions = interactions.set_index('user_id') \
            .join(users_df[['user_id', 'sex', 'age', 'income']]
                .set_index('user_id'))
        interactions.reset_index(inplace=True)
        sex_stats = interactions.groupby('item_id')['sex'] \
            .value_counts(normalize=True)
        sex_stats = pd.DataFrame(sex_stats)
        sex_stats.columns = ['value']
        sex_stats.reset_index(inplace=True)
        sex_stats.columns = ['item_id', 'sex', 'value']
        sex_stats = sex_stats.pivot(index='item_id',
                                    columns='sex',
                                    values='value').drop('sex_unknown', axis=1)
        sex_stats.fillna(0, inplace=True)
        item_stats = item_stats.set_index('item_id').join(sex_stats)
        item_stats[['F', 'M']] = item_stats[['F', 'M']].fillna(0)
        item_stats.rename(columns={'F': 'female_watchers_fraction',
                                'M': 'male_watchers_fraction'},
                        inplace=True)
        return item_stats

    @staticmethod
    def get_coo_matrix(df,
                    user_col='user_id',
                    item_col='item_id',
                    weight_col=None,
                    users_mapping={},
                    items_mapping={}):
        if weight_col is None:
            weights = np.ones(len(df), dtype=np.float32)
        else:
            weights = df[weight_col].astype(np.float32)

        interaction_matrix = sp.coo_matrix((
            weights,
            (
                df[user_col].map(users_mapping.get),
                df[item_col].map(items_mapping.get)
            )
        ))
        return interaction_matrix

    @staticmethod
    def create_mapping():
        # Creating items and users mapping
        users_inv_mapping = dict(enumerate(df['user_id'].unique()))
        users_mapping = {v: k for k, v in users_inv_mapping.items()}
        items_inv_mapping = dict(enumerate(df['item_id'].unique()))
        items_mapping = {v: k for k, v in items_inv_mapping.items()}

    def process(self):
        # load data
        print('Loading data')
        users_df = self.load_users_df()
        items_df = self.load_items_df()
        interactions_df = self.load_ratings_df()

        # users info preprocessing
        print('Processing users info')
        users_df['age'] = users_df['age'].fillna('age_unknown')
        users_df['age'] = users_df['age'].astype('category')
        users_df['income'] = users_df['income'].fillna('income_unknown')
        users_df['income'] = users_df['income'].astype('category')
        users_df['sex'] = users_df['sex'].fillna('sex_unknown')
        users_df.loc[users_df.sex == 'М', 'sex'] = 'M'
        users_df.loc[users_df.sex == 'Ж', 'sex'] = 'F'
        users_df['sex'] = users_df['sex'].astype('category')
        users_df['kids_flg'] = users_df['kids_flg'].astype('bool')

        # items info preprocessing
        print('Processing items info')
        items_df['content_type'] = items_df['content_type'].astype('category')
        items_df['title'] = items_df['title'].str.lower()
        items_df['title_orig'] = items_df['title_orig'].fillna('None')
        items_df.loc[items_df['release_year'] < 1980, 'release_novelty'] = 1
        items_df.loc[items_df['release_year'] >= 2020, 'release_novelty'] = 6
        novelty = 1
        for i in range(1980, 2020, 10):
            novelty += 1
            items_df.loc[(items_df['release_year'] >= i) &
                        (items_df['release_year'] < i + 10), 'release_novelty'] = novelty
        items_df = items_df.drop(columns=['release_year'])
        items_df['for_kids'] = items_df['for_kids'].fillna(0)
        items_df['for_kids'] = items_df['for_kids'].astype('bool')
        items_df.loc[items_df.age_rating.isna(), 'age_rating'] = 0
        items_df['age_rating'] = items_df['age_rating'].astype('category')
        items_df['genres_list'] = items_df['genres'].apply(lambda x: x.split(', '))
        num_genres = pd.Series(np.hstack(items_df['genres_list'].values)).value_counts()
        items_df['genres_min'] = items_df['genres_list'].apply(
            lambda x: min([num_genres[el] for el in x]))
        items_df['genres_max'] = items_df['genres_list'].apply(
            lambda x: max([num_genres[el] for el in x]))
        items_df['genres_med'] = items_df['genres_list'].apply(
            lambda x: (np.median([num_genres[el] for el in x])))
        items_df['countries'].fillna('None', inplace=True)
        items_df['countries'] = items_df['countries'].str.lower()
        items_df['countries_list'] = items_df['countries'].apply(
            lambda x: x.split(', ') if ', ' in x else [x])
        num_countries = pd.Series(np.hstack(items_df['countries_list'].values)).value_counts()
        items_df['countries_max'] = items_df['countries_list'].apply(
            lambda x: max([num_countries[el] for el in x]))
        items_df['studios'].fillna('None', inplace=True)
        items_df['studios'] = items_df['studios'].str.lower()
        items_df['studios_list'] = items_df['studios'].apply(
            lambda x: x.split(', ') if ', ' in x else [x])
        num_studios = pd.Series(np.hstack(items_df['studios_list'].values)).value_counts()
        items_df['studios_max'] = items_df['studios_list'].apply(
            lambda x: max([num_studios[el] for el in x]))
        items_df.drop(['countries_list', 'genres_list', 'studios_list'],
                    axis=1, inplace=True)
        
        # interactions preprocessing
        print('Processing interactions')
        interactions_df['watched_pct'] = interactions_df['watched_pct'].astype(
            pd.Int8Dtype())
        interactions_df['watched_pct'] = interactions_df['watched_pct'].fillna(0)
        interactions_df['last_watch_dt'] = pd.to_datetime(
            interactions_df['last_watch_dt'])
        interactions_df.sort_values(by='last_watch_dt', inplace=True)
        
        # user stats feature engineering
        print('Processing users stats')
        max_date = interactions_df['last_watch_dt'].max()
        boosting_split_date = max_date - pd.Timedelta(days=14)
        interactions_boost = interactions_df[
            interactions_df['last_watch_dt'] <= boosting_split_date]
        users_df = self.add_user_stats(interactions_boost, users_df, split_name='boost_')
        users_df = self.add_user_stats(interactions_df, users_df, split_name='')

        # Item stats
        print('Processing items stats')
        item_stats = items_df[['item_id']]
        item_stats = item_stats.set_index('item_id')
        item_stats = self.add_item_watches_stats(interactions_boost, items_df, item_stats)
        item_stats.fillna(0, inplace=True)
        item_stats = self.add_sex_stats(interactions_boost, item_stats, users_df)
        item_stats = self.add_age_stats(interactions_boost, item_stats, users_df)

        # Saving preprocessed files
        users_df.to_csv(self.processed_paths[0], index=False)
        items_df.to_csv(self.processed_paths[1], index=False)
        interactions_df.to_csv(self.processed_paths[2], index=False)
        item_stats.to_csv(self.processed_paths[3], index=True)

Example

In [ ]:
#hide
!apt-get -qq install tree
!pip install -q watermark
!pip install -q pytorch-lightning

In [ ]:
ds = MTSDataset(data_dir='/content/data')

Processing...


Loading data
Processing users info
Processing items info
Processing interactions
Processing users stats
Processing items stats


Done!


In [ ]:
!tree --du -h -C ./data

./data
├── [250M]  processed
│   ├── [161M]  interactions_processed.csv
│   ├── [ 31M]  items_processed.csv
│   ├── [1.5M]  item_stats_for_boost_train.csv
│   ├── [1.5M]  item_stats_for_submit.csv
│   └── [ 55M]  users_processed.csv
└── [ 77M]  raw
    ├── [ 56M]  interactions.parquet.snappy
    ├── [ 15M]  items.parquet.snappy
    └── [5.1M]  users.parquet.snappy

 327M used in 2 directories, 8 files


> **References**
> - https://ods.ai/tracks/recsys-course2021/competitions/competition-recsys-21
> - https://github.com/blondered/ods_MTS_RecSys_Challenge_solution

In [ ]:
#hide
%reload_ext watermark
%watermark -a "Sparsh A." -m -iv -u -t -d -p recohut

Author: Sparsh A.

Last updated: 2022-01-14 10:32:26

recohut: 0.0.11

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 5.4.144+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit

IPython: 5.5.0
pandas : 1.1.5
numpy  : 1.19.5

